In [4]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 

import polars.selectors as cs
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as mn

# dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"
dataPath = "../data/"

In [5]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

# Load feature_def

In [6]:
def get_feature_definitions(columns):
    return pl.DataFrame({'Variable': columns}).join(
        feature_def,
        on = 'Variable',
        how = 'left',
    )

feature_def = pl.read_csv(dataPath + "feature_definitions.csv")

# Load Basetable

In [7]:
# train
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")

# test
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")

# Check Basetable

In [ ]:
# print(train_basetable.columns)
# print('\n')
# print(train_basetable.shape)
# print('\n')
# display(train_basetable.head())

# Load depth=0

In [8]:
# train
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)

# test
test_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)

# Check depth=0

In [ ]:
# print(train_static.columns)
# print('\n')
# print(train_static.shape)
# print('\n')
# display(train_static.head())

In [ ]:
# print(train_static_cb.columns)
# print('\n')
# print(train_static_cb.shape)
# print('\n')
# display(train_static_cb.head())

# depth=1에 해당하는 테이블

Internal file
> `applprev_1` `debitcard_1` `deposit_1` `other_1` `person_1`

External file
> `credit_bureau_b_1` 

In [9]:
## applprev_1
# train
train_applprev_1 = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_applprev_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + 'csv_files/train/train_applprev_1_1.csv').pipe(set_table_dtypes),
    ],
    how='vertical_relaxed',
)

# test
test_applprev_1 = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_applprev_1_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + 'csv_files/test/test_applprev_1_1.csv').pipe(set_table_dtypes),
        pl.read_csv(dataPath + 'csv_files/test/test_applprev_1_2.csv').pipe(set_table_dtypes),
    ],
    how='vertical_relaxed',
)

In [10]:
# train
train_debitcard_1 = pl.read_csv(dataPath + "csv_files/train/train_debitcard_1.csv").pipe(set_table_dtypes) 
train_deposit_1 = pl.read_csv(dataPath + "csv_files/train/train_deposit_1.csv").pipe(set_table_dtypes)
train_other_1 = pl.read_csv(dataPath + 'csv_files/train/train_other_1.csv').pipe(set_table_dtypes)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes)
train_credit_bureau_b_1 = pl.read_csv(dataPath + 'csv_files/train/train_credit_bureau_b_1.csv').pipe(set_table_dtypes)

# test
test_debitcard_1 = pl.read_csv(dataPath + "csv_files/test/test_debitcard_1.csv").pipe(set_table_dtypes)
test_deposit_1 = pl.read_csv(dataPath + "csv_files/test/test_deposit_1.csv").pipe(set_table_dtypes)
test_other_1 = pl.read_csv(dataPath + 'csv_files/test/test_other_1.csv').pipe(set_table_dtypes)
test_person_1 = pl.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes)
test_credit_bureau_b_1 = pl.read_csv(dataPath + 'csv_files/test/test_credit_bureau_b_1.csv').pipe(set_table_dtypes)

In [11]:
print(
    train_applprev_1.shape,
    train_debitcard_1.shape, 
    train_deposit_1.shape, 
    train_other_1.shape, 
    train_person_1.shape,
    train_credit_bureau_b_1.shape,
)

(6525979, 41) (157302, 6) (145086, 5) (51109, 7) (2973991, 37) (85791, 45)


# Feature Engineering

In [12]:
## applprev_1
app_cols = [
    'case_id',
    'maxdpdtolerance_577P',
    'employedfrom_700D',
    'pmtnum_8L',
    'creationdate_885D',
    'credamount_590A',
    'mainoccupationinc_437A',
    'firstnonzeroinstldate_307D',
    'dtlastpmtallstes_3545839D',
    'annuity_853A',
    'familystate_726L',
    'approvaldate_319D',
    'district_544M',
    'dateactivated_425D',
    'education_1138M',
]

# train
train_applprev_1 = train_applprev_1[app_cols]
train_applprev_1 = train_applprev_1.unique(subset='case_id', keep='first')

# test
test_applprev_1 = test_applprev_1[app_cols]
test_applprev_1 = test_applprev_1.unique(subset='case_id', keep='first')

In [13]:
## case_id를 기준으로 그룹화한 후, aggregation functions

# debitcard_1
train_debitcard_1_feats = train_debitcard_1.group_by("case_id").agg(
    # 체크카드의 평균 잔액의 최대값
    pl.col('last180dayaveragebalance_704A').max().alias('max_last180dayaveragebalance_704A'),
    # 180일간의 체크카드 거래액의 최대값
    pl.col('last180dayturnover_1134A').max().alias('max_last180dayturnover_1134A'),
    # 30일간의 체크카드 거래액의 최대값
    pl.col("last30dayturnover_651A").max().alias("max_last30dayturnover_651A"),
).sort(by='case_id')

# deposit_1
train_deposit_1_feats = train_deposit_1.group_by('case_id').agg(
    # 예/적금의 합계
    pl.sum('amount_416A').alias('sum_amount_416A'),
).sort(by='case_id')

# other_1
train_other_1_feats = train_other_1.group_by("case_id").agg(
    # 입금액의 합계
    pl.col("amtdepositincoming_4809444A").sum().alias("sum_amtdepositincoming_4809444A"),
    # 출금액의 합계
    pl.col("amtdepositoutgoing_4809442A").sum().alias("sum_amtdepositoutgoing_4809442A"),
).sort(by='case_id')

# person_1
train_person_1_feats_1 = train_person_1.select(['case_id', 'incometype_1044T','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

train_person_1_feats_2 = train_person_1.select(['case_id', 'language1_981M','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

# credit_bureau_b_1
train_credit_bureau_b_1_feats = train_credit_bureau_b_1.group_by('case_id').agg(
    pl.col('installmentamount_644A').mean().alias('mean_installmentamount_644A'),
    pl.col('maxdebtpduevalodued_3940955A').mean().alias('mean_maxdebtpduevalodued_3940955A'),
    pl.col('overdueamountmax_950A').mean().alias('mean_overdueamountmax_950A'),
    pl.col('residualamount_1093A').mean().alias('mean_residualamount_1093A'),
).sort(by='case_id')


## ⭐키워드 6가지 중 어떤걸 선택할지?
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "D", "M"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "D", "M"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)


## Join
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_applprev_1, how="left", on="case_id"
).join(
    train_debitcard_1_feats, how="left", on="case_id"
).join(
    train_deposit_1_feats, how="left", on="case_id"
).join(
    train_other_1_feats, how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_1_feats, how="left", on="case_id"
)


## num_group1이 0인 경우 대출을 신청한 사람
# train_debitcard_1_feats_2 = train_debitcard_1.select(["case_id", "num_group1"]).filter(
#     pl.col("num_group1") == 0 # 필터링 후
# ).drop("num_group1") # 컬럼 삭제

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'datefirstoffer_1144D', 'datelastinstal40dpd_247D', 'datelastunpaid_3546854D', 'disbursedcredamount_1113A', 'downpmt_116A', 'dtlastpmtallstes_4499206D', 'firstclxcampaign_1125D', 'firstdatedue_489D', 'inittransactionamount_650A', 'lastactivateddate_801D', 'lastapplicationdate_877D', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastapprdate_640D', 'lastcancelreason_561M', 'lastdelinqdate_224D', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectdate_50D', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastrepayingdate_696D', 'maininc_215A', 'maxannuity_159A', 'maxannuity_

In [ ]:
test_debitcard_1_feats = test_debitcard_1.group_by("case_id").agg(
    pl.col('last180dayaveragebalance_704A').max().alias('max_last180dayaveragebalance_704A'),
    pl.col('last180dayturnover_1134A').max().alias('max_last180dayturnover_1134A'),
    pl.col("last30dayturnover_651A").max().alias("max_last30dayturnover_651A"),
).sort(by='case_id')

test_deposit_1_feats = test_deposit_1.group_by('case_id').agg(
    pl.sum('amount_416A').alias('sum_amount_416A')
).sort(by='case_id')

test_other_1_feats = test_other_1.group_by("case_id").agg(
    pl.col("amtdepositincoming_4809444A").sum().alias("sum_amtdepositincoming_4809444A"),
    pl.col("amtdepositoutgoing_4809442A").sum().alias("sum_amtdepositoutgoing_4809442A"),
).sort(by='case_id')

test_person_1_feats_1 = test_person_1.select(['case_id', 'incometype_1044T','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

test_person_1_feats_2 = test_person_1.select(['case_id', 'language1_981M','num_group1']).filter(
    pl.col('num_group1') == 0
).drop('num_group1')

test_credit_bureau_b_1_feats = test_credit_bureau_b_1.group_by('case_id').agg(
    pl.col('installmentamount_644A').mean().alias('mean_installmentamount_644A'),
    pl.col('maxdebtpduevalodued_3940955A').mean().alias('mean_maxdebtpduevalodued_3940955A'),
    pl.col('overdueamountmax_950A').mean().alias('mean_overdueamountmax_950A'),
    pl.col('residualamount_1093A').mean().alias('mean_residualamount_1093A'),
).sort(by='case_id')


data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    test_applprev_1, how="left", on="case_id"
).join(
    test_debitcard_1_feats, how="left", on="case_id"
).join(
    test_deposit_1_feats, how="left", on="case_id"
).join(
    test_other_1_feats, how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_1_feats, how="left", on="case_id"
)

In [14]:
case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)
    
print('\n')
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'datefirstoffer_1144D', 'datelastinstal40dpd_247D', 'datelastunpaid_3546854D', 'disbursedcredamount_1113A', 'downpmt_116A', 'dtlastpmtallstes_4499206D', 'firstclxcampaign_1125D', 'firstdatedue_489D', 'inittransactionamount_650A', 'lastactivateddate_801D', 'lastapplicationdate_877D', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastapprdate_640D', 'lastcancelreason_561M', 'lastdelinqdate_224D', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectdate_50D', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastrepayingdate_696D', 'maininc_215A', 'maxannuity_159A', 'maxannuity_

## Optuna

In [18]:
lgb_train = lgb.Dataset(X_train, label=y_train, params={'feature_pre_filter': False})
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train, params={'feature_pre_filter': False})

In [19]:
def gini_stability_metric(base, y_pred):
    base['score'] = y_pred
    gini_scores = base.groupby('WEEK_NUM').apply(lambda x: 2 * roc_auc_score(x['target'], x['score']) - 1).tolist()

    x = np.arange(len(gini_scores))
    y = np.array(gini_scores)
    a, b = np.polyfit(x, y, 1)
    residuals = y - (a * x + b)
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_scores)

    stability_score = avg_gini + 88.0 * min(0, a) - 0.5 * res_std
    return stability_score

def objective(trial):
    param = {
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "max_depth": trial.suggest_int("max_depth", 10, 35),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2 ** trial.suggest_int("max_depth_log2", 3, 5)),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 200),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.8),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "is_unbalanced": True,
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
    }
    
    gbm = lgb.train(
        param,
        lgb_train,
        num_boost_round=trial.suggest_int("n_estimators", 200, 50000),
        valid_sets=[lgb_valid],
        callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
    )

    preds = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
    # AUC 점수는 여기서 계산하지만, 최적화 목적으로 사용하지 않습니다.
    # auc = roc_auc_score(y_valid, preds)
    
    # "gini stability metric" 계산 및 반환
    stability_score = gini_stability_metric(base_valid, preds)
    return stability_score

In [20]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-02-21 11:15:00,037] A new study created in memory with name: no-name-5fc9a75a-b04b-4030-9822-dce186ecf510


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.763146
[100]	valid_0's auc: 0.777784
[150]	valid_0's auc: 0.784815
[200]	valid_0's auc: 0.78853
[250]	valid_0's auc: 0.790995
[300]	valid_0's auc: 0.792555
[350]	valid_0's auc: 0.793646
[400]	valid_0's auc: 0.794388
[450]	valid_0's auc: 0.794893
[500]	valid_0's auc: 0.795304
[550]	valid_0's auc: 0.795818
[600]	valid_0's auc: 0.796212
[650]	valid_0's auc: 0.796697
[700]	valid_0's auc: 0.796984
[750]	valid_0's auc: 0.797498
Early stopping, best iteration is:
[751]	valid_0's auc: 0.7975


[I 2024-02-21 11:15:40,604] Trial 0 finished with value: 0.5651111676464874 and parameters: {'max_depth': 18, 'max_depth_log2': 3, 'num_leaves': 8, 'min_child_samples': 12, 'colsample_bytree': 0.6027761810285048, 'learning_rate': 0.06150669175028314, 'reg_alpha': 0.5143072959209114, 'reg_lambda': 0.6320432230166775, 'n_estimators': 25306}. Best is trial 0 with value: 0.5651111676464874.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's auc: 0.680297


[I 2024-02-21 11:15:42,310] Trial 1 finished with value: 0.326865362153303 and parameters: {'max_depth': 15, 'max_depth_log2': 4, 'num_leaves': 4, 'min_child_samples': 148, 'colsample_bytree': 0.5046996434028499, 'learning_rate': 0.0023074064773134506, 'reg_alpha': 0.9485576591015746, 'reg_lambda': 0.6077371127673352, 'n_estimators': 3455}. Best is trial 0 with value: 0.5651111676464874.


...

Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.758819
[100]	valid_0's auc: 0.769915
[150]	valid_0's auc: 0.777631
[200]	valid_0's auc: 0.782847
[250]	valid_0's auc: 0.786488
[300]	valid_0's auc: 0.789642
[350]	valid_0's auc: 0.791956
[400]	valid_0's auc: 0.793823
[450]	valid_0's auc: 0.795342
[500]	valid_0's auc: 0.796508
[550]	valid_0's auc: 0.797445
[600]	valid_0's auc: 0.798301
[650]	valid_0's auc: 0.798833
[700]	valid_0's auc: 0.799312
[750]	valid_0's auc: 0.799667
[800]	valid_0's auc: 0.799927
[850]	valid_0's auc: 0.800135
[900]	valid_0's auc: 0.800403
[950]	valid_0's auc: 0.800634
[1000]	valid_0's auc: 0.800834
[1050]	valid_0's auc: 0.801058
[1100]	valid_0's auc: 0.801212
[1150]	valid_0's auc: 0.80143
[1200]	valid_0's auc: 0.801595
[1250]	valid_0's auc: 0.801709
[1300]	valid_0's auc: 0.801824
[1350]	valid_0's auc: 0.801907
[1400]	valid_0's auc: 0.801993
[1450]	valid_0's auc: 0.802108
[1500]	valid_0's auc: 0.802214
[1550]	valid_0's auc: 0.802293

[I 2024-02-21 13:13:35,593] Trial 48 finished with value: 0.5760047285577686 and parameters: {'max_depth': 32, 'max_depth_log2': 5, 'num_leaves': 31, 'min_child_samples': 114, 'colsample_bytree': 0.6015426024352372, 'learning_rate': 0.01557848230768421, 'reg_alpha': 0.7244879069197534, 'reg_lambda': 0.9636208081294657, 'n_estimators': 36654}. Best is trial 48 with value: 0.5760047285577686.


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.74181
[100]	valid_0's auc: 0.756217
[150]	valid_0's auc: 0.766384
[200]	valid_0's auc: 0.773172
[250]	valid_0's auc: 0.777594
[300]	valid_0's auc: 0.781105
[350]	valid_0's auc: 0.783847
[400]	valid_0's auc: 0.786095
[450]	valid_0's auc: 0.788009
[500]	valid_0's auc: 0.78956
[550]	valid_0's auc: 0.790791
[600]	valid_0's auc: 0.791864
[650]	valid_0's auc: 0.792752
[700]	valid_0's auc: 0.793538
[750]	valid_0's auc: 0.794163
[800]	valid_0's auc: 0.79475
[850]	valid_0's auc: 0.795305
[900]	valid_0's auc: 0.79575
[950]	valid_0's auc: 0.796079
[1000]	valid_0's auc: 0.796429
[1050]	valid_0's auc: 0.796713
[1100]	valid_0's auc: 0.796893
[1150]	valid_0's auc: 0.797108
[1200]	valid_0's auc: 0.79733
[1250]	valid_0's auc: 0.797492
[1300]	valid_0's auc: 0.797736
[1350]	valid_0's auc: 0.797889
[1400]	valid_0's auc: 0.798068
[1450]	valid_0's auc: 0.798203
[1500]	valid_0's auc: 0.798354
[1550]	valid_0's auc: 0.798506
[16

[I 2024-02-21 13:15:10,702] Trial 49 finished with value: 0.5694320543714997 and parameters: {'max_depth': 32, 'max_depth_log2': 4, 'num_leaves': 13, 'min_child_samples': 178, 'colsample_bytree': 0.5872964186985181, 'learning_rate': 0.01596905281363398, 'reg_alpha': 0.72450627189425, 'reg_lambda': 0.1144805301290377, 'n_estimators': 36261}. Best is trial 48 with value: 0.5760047285577686.


Number of finished trials:  50
Best trial:
  Value: 0.5760047285577686
  Params: 
    max_depth: 32
    max_depth_log2: 5
    num_leaves: 31
    min_child_samples: 114
    colsample_bytree: 0.6015426024352372
    learning_rate: 0.01557848230768421
    reg_alpha: 0.7244879069197534
    reg_lambda: 0.9636208081294657
    n_estimators: 36654


# Training LightGBM

Minimal example of LightGBM training is shown below.

In [ ]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

Evaluation with AUC and then comparison with the stability metric is shown below.

In [ ]:
for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')  


def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)
stability_score_test = gini_stability(base_test)

print('\n')
print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}')

# Submission

Scoring the submission dataset is below, we need to take care of new categories. Then we save the score as a last step. 

In [ ]:
# # 데이터 타입 일치 여부 확인
# dtype_match = X_train.dtypes == X_submission.dtypes

# if dtype_match.all():
#     print("데이터 타입이 일치합니다.")
# else:
#     print("데이터 타입이 일치하지 않습니다.")
    
#     # 불일치한 열과 데이터 타입 출력
#     mismatched_cols = X_train.columns[~dtype_match].tolist()
#     for col in mismatched_cols:
#         print(f"데이터 타입 불일치 - 열: {col}, 훈련 데이터셋: {X_train[col].dtype}, 제출 데이터셋: {X_submission[col].dtype}")

In [ ]:
X_submission = data_submission[cols_pred].to_pandas()
X_submission = convert_strings(X_submission)
categorical_cols = X_train.select_dtypes(include=['category']).columns

for col in categorical_cols:
    train_categories = set(X_train[col].cat.categories)
    submission_categories = set(X_submission[col].cat.categories)
    new_categories = submission_categories - train_categories
    X_submission.loc[X_submission[col].isin(new_categories), col] = "Unknown"
    new_dtype = pd.CategoricalDtype(categories=train_categories, ordered=True)
    X_train[col] = X_train[col].astype(new_dtype)
    X_submission[col] = X_submission[col].astype(new_dtype)

y_submission_pred = gbm.predict(X_submission, num_iteration=gbm.best_iteration)


submission = pd.DataFrame({
    "case_id": data_submission["case_id"].to_numpy(),
    "score": y_submission_pred
}).set_index('case_id')
submission.to_csv("./submission.csv")